# Семинар 10

In [ ]:
import sys, os

if "google.colab" in sys.modules and not os.path.exists(".setup_complete"):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ["DISPLAY"] = ":1"

Starting virtual X frame buffer: Xvfb.


In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque, defaultdict
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython.display import clear_output, display

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tabular Q-learning

Реализуем алгоритм Q-learning для среды CliffWalking. 

In [ ]:
from gym.envs import toy_text

In [ ]:
env = toy_text.CliffWalkingEnv()

![](https://drive.google.com/uc?export=view&id=16nbZM2ZoY7xabN4uDEvCCz9IWmk_9D_x)

In [ ]:
print(env.__doc__)


    This is a simple implementation of the Gridworld Cliff
    reinforcement learning task.

    Adapted from Example 6.6 (page 106) from Reinforcement Learning: An Introduction
    by Sutton and Barto:
    http://incompleteideas.net/book/bookdraft2018jan1.pdf

    With inspiration from:
    https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/cliff_walking.py

    The board is a 4x12 matrix, with (using Numpy matrix indexing):
        [3, 0] as the start at bottom-left
        [3, 11] as the goal at bottom-right
        [3, 1..10] as the cliff at bottom-center

    Each time step incurs -1 reward, and stepping into the cliff incurs -100 reward
    and a reset to the start. An episode terminates when the agent reaches the goal.
    


Так как награда выражатеся отрицательными значениями, то фактической целью агента является как можно более быстрое преодоление пути от старта к финишу при этом ему нужно не упасть с обрыва.

Традиционно посмотрим сколько очков в среднем за 100 эпизодов сможет набрать "cлучайный" агент.

In [ ]:
total_reward = []
for episode in range(10):
    
    for t in range(100):
        
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

In [ ]:
print(np.mean(total_reward))

Ожидаемо, наш "случайный" агент просто блуждает по среде, не пытаясь добраться до цели.

Вспомним как выглядит реализация данного алгоритма 
![](https://drive.google.com/uc?export=view&id=1eMG5HUaDyKKE8DQUhEdOaBZVR1UqOTO9)

Так как мы раелизуем табличную версию Q-learning, создадим структура, которая будет хранить значения нашей функции *Q(S,A)* для каждого состояния и действия. Она пдетставляет собой словарь (*dict*), хранящий в качестве ключей состояния, а в качестве значений массив значений *Q-функции* для каждого действия для данного ключа-состояния.

In [ ]:
Q = defaultdict(lambda: np.zeros(env.action_space.n))

print("Q-значения для состояния-действия (0, 0): %s" % Q[(0, 0)], "хранятся в списке, по значению для каждого действия")
print("Таким обарзом, Q-значение для действия 3 в в состоянии (1,2), i.e. Q((1,2), 3), можно получить вот так q_vals[(1,2)][3]:", Q[(1,2)][3])

Q-значения для состояния-действия (0, 0): [0. 0. 0. 0.] хранятся в списке, по значению для каждого действия
Таким обарзом, Q-значение для действия 3 в в состоянии (1,2), i.e. Q((1,2), 3), можно получить вот так q_vals[(1,2)][3]: 0.0


Сначала напишем функцию реализующую $\epsilon$-greedy политику для исследования среды.

In [ ]:
def eps_greedy(Q, state, epsilon):
    """
    Параметры:
        Q: таблица значений Q-функции
        epsilon: параметр эпсилон
        state: текущее состоние
    Результат:
        Случайное действие с вероятностью eps или argmax Q(s, .) c вероятностью (1 - eps)
    """
    # To Do



Также напишем функцию, которая поможет нам оценить, как ведет себя обученный агент.

In [ ]:
def q_eval(Q, render=False):
    total_reward = []
    for i_episode in range(10):
        episode_reward = 0
        observation = env.reset()
        for t in range(100):
            
            
            
            observation, reward, done, info = env.step(action)
            episode_reward += reward

        total_reward.append(episode_reward)
    return np.mean(total_reward)

Теперь реализуем функцию обучения агента (обновления значений Q-функции) с использование функции, реализующей $\epsilon$-greedy политику.

In [ ]:
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """    
    Параменты:
        env: среда Open AI
        num_episodes: Количество эпизодов
        discount_factor: фактор дисконтирования
        alpha: константа обучения
        epsilon: параметр эпсилон для ϵ-greedy политику
    
    Результат:
        Таблица с оптимальными значениями Q-функции
    """
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    episode_lengths=np.zeros(num_episodes)
    episode_rewards=np.zeros(num_episodes)
        
    for i_episode in range(num_episodes):
        if (i_episode + 1) % 100 == 0:
            print("\rEpisode {}/{} score = {}.".format(i_episode + 1, num_episodes, q_eval(Q)))
        
        state = env.reset()
        done = False
        len = 0
        # To Do
          
        # While episode is not over

    
    return Q, episode_lengths, episode_rewards

In [ ]:
Q, episode_lengths, episode_rewards = q_learning(env, 500)

In [ ]:
score = q_eval(Q)

In [ ]:
plt.plot(episode_lengths)

In [ ]:
plt.plot(episode_rewards)

# DQN

## Q-learning algorithm

Наша цель будет состоять в обучении политике, которая пытается максимизировать дисконтированные,
накопительное вознаграждение
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$

$Q^*: State \times Action \rightarrow \mathbb{R}$

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

Обновляем Q функцию исходя из того, что политика должна удовлетворять уравнению Беллмана

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

Разницу между правой и левой частями уравнения называем temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}

Именно эту ошибку мы и будем мимнимизировать

## Будем работать непосредственно с экраном игры, для этого объявим вспомогательные функции

In [ ]:
env = gym.make('CartPole-v0').unwrapped

Преобразование картинки в тензор

In [ ]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Считывание положения каретки

In [ ]:
def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)

Считывание экрана игры

In [ ]:
def get_screen():
    # gym возвращает экран с размером 400x600x3
    # Transpose it into torch order (CHW).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    
    # Обрежем верх и низ картинки
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    
    # Центрируем картинку по каретке
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
        
    # Обрезаем лишнюю часть, чтобы получить квадратную область
    screen = screen[:, :, slice_range]
    
    # Переводим картинку в тензор
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    
    # Сжимаем картинку и добавляем размерность батча
    return resize(screen).unsqueeze(0)

In [ ]:
env.reset()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(), interpolation='none')
plt.title('Example extracted screen')
plt.show()

## Replay Memory

Мы будем использовать воспроизведения опыта для тренировки нашей DQN. Replay Memory хранит
переходы, которые наблюдает агент, что позволяет нам повторно использовать эти данные
потом. Для уменьшения корреляций между состояниями и переходами мы будем выбирать случайные переходы из предыдущих игр.

In [ ]:
# Создаем кортежи, в которых будем хранить наш опыт
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

# Создаем буфер, в котором будем хранить наш опыт
class ReplayMemory(object):

    def __init__(self, capacity):
       self.memory = deque([],maxlen=capacity)        

    def push(self, *args):
        """Saves a transition."""
        self.memory.append(Transition(*args))        

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size) 

    def __len__(self):
        return len(self.memory)

## Создадим модель для апроксимации политики

In [ ]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
      super(DQN, self).__init__()


      def conv2d_size_out(size, kernel_size = 5, stride = 2):
          return (size - (kernel_size - 1) - 1) // stride  + 1



    def forward(self, x):

      return self.head(x.view(x.size(0), -1))

## Training

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 400
TARGET_UPDATE = 10

In [ ]:
for i in range(1500):
  print(math.exp(-1. * i / EPS_DECAY))

In [ ]:
init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape

n_actions = env.action_space.n

Объявляем две модели, одна отвечает за политику во время игры, другая для предсказания награды

In [ ]:
policy_net = 

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(100000)

episode_durations = []

In [ ]:
steps_done = 0

Выбор действия

In [ ]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    print(eps_threshold)
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

Визуализация

In [ ]:
def plot_durations():
    clear_output(wait=True)
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 30:
        means = durations_t.unfold(0, 30, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(29), means))
        plt.plot(means.numpy())

    plt.pause(0.0001)  # pause a bit so that plots are updated

## Training loop

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return

    transitions = memory.sample(BATCH_SIZE)
    
    # batch-array of Transitions to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Парсим transitions чтобы сформировать обучающую выборку
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Вычисляем Q(s_t, a) - Модель предсказывает Q(s_t), 
    # затем мы выбираем Q, соответствующие сделанному действию
    state_action_values = #

    # Вычисляем V(s_{t+1}) для всех следующих состояний target_net(non_final_next_states)
    # берем максимальной значение
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = #
    
    # Вычисляем ожидаемые значения Q функции
    expected_state_action_values = #

    # Вычисляем Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion()

    # Шаг оптимизации
    optimizer.zero_grad()
    loss.backward()
    #for param in policy_net.parameters():
    #    param.grad.data.clamp_(-1, 1)
    optimizer.step()

## Основной цикл обучения




In [ ]:
num_episodes = 50
for i_episode in range(num_episodes):
    
    # Инициализация
    env.reset()
    last_screen = get_screen()
    current_screen = get_screen()
    state = current_screen - last_screen    
    
    for t in count():
        
        # Выбираем действие
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Наблюдаем новое состояние
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Сохраняем transition в буфер
        memory.push(state, action, next_state, reward)
        state = next_state

        # Делаем шаг оптимизации
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
            
    # Обновляем target network, копирую веса из policy_net
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

In [ ]:
from IPython.display import clear_output, display
import time

env.reset()
last_screen = get_screen()
current_screen = get_screen()
state = current_screen - last_screen    

time_limit = 80
for time_step in range(time_limit):
    # Choose action based on your policy.
    action = policy_net(state).max(1)[1].view(1, 1)
    _, reward, done, _ = env.step(action.item())
    print(action.item())

    # Наблюдаем новое состояние
    last_screen = current_screen
    current_screen = get_screen()
    if not done:
        next_state = current_screen - last_screen
    else:
        next_state = None

    state = next_state

    # Draw game image on display.
    clear_output(wait=True)
    plt.title(time_step)
    plt.imshow(env.render("rgb_array"))
    plt.show()
    time.sleep(.5)

    if time_step > 79:
        print("Well done!")
        break

    if done:
        print("you lose")
        break